In [ ]:
# !python -m venv venv
# # !venv/scripts/activate
%pip install pandas numpy matplotlib seaborn plotly geopandas scipy scikit-learn statsmodels requests aiohttp shapely ipython h3 ipywidgets dtale tqdm pickle

In [1]:

import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import numpy as np

# Data Visualization
import matplotlib.pyplot as plt

import requests
import os

import datetime as dt

from pandas import read_csv
import concurrent.futures
import psutil

from datetime import datetime

import h3
import geopandas as gpd
import shapely
import plotly.express as px

from tqdm import tqdm
tqdm.pandas()

from pandas import read_pickle
import matplotlib.dates as mdates

import seaborn as sn
import sys


In [ ]:
# Verify that the hexagon grid is suitable
# create_choropleth_map_only_hexagon_df(geojson_df=hexagon_df)

# def create_choropleth_map_only_hexagon_df(geojson_df, alpha=0.1, map_style="carto-positron", color_scale="Viridis"):
#     """
#     Create an interactive choropleth map using Plotly Express.
#     Parameters:
#     - geojson_df (GeoDataFrame): GeoJSON data containing polygon geometries.
#     - alpha (float): Opacity level for the map polygons (0.0 to 1.0).
#     - map_style (str): Map style for the Plotly map (e.g., "carto-positron").
#     - color_scale (str): Color scale for the choropleth map.
#     Returns:
#     None
#     """
#     # Create a choropleth map using px.choropleth_mapbox
#     fig = px.choropleth_mapbox(
#         geojson_df,
#         geojson=geojson_df.geometry,
#         locations=geojson_df.index,  # Use index as locations to avoid duplicate rows
#         # color="Count",
#         color_continuous_scale=color_scale,
#         title="GPS Jam Map",
#         mapbox_style=map_style,
#         center={"lat": home_lat, "lon": home_lng},  # Adjust the center as needed
#         zoom=2,
#     )

#     # Customize the opacity of the hexagons
#     fig.update_traces(marker=dict(opacity=alpha))

#     # Set margins to 25 on all sides
#     fig.update_layout(margin=dict(l=35, r=35, t=45, b=35))
    
#     # Adjust the width of the visualization
#     fig.update_layout(width=1000) 

#     fig.show()


In [2]:



# Latitude and longitude coordinates
home_lat = 1.3472764
home_lng = 103.9104234

# Generate H3 hexagons at a specified resolution (e.g., 9)
resolution = 5

# Indicate the number of rings around the central hexagon
ring_size = 463

# global dataframes to avoid keep loading files
df1=None
df2=None
flights_data=None
hexagon_df=None

def calculate_hexagon_ids(df):
    """
    Calculate Hexagon IDs for each row(ping) in a DataFrame based on their geographic coordinates.
    Args:
        df (pd.DataFrame): DataFrame containing ADSB data with "lat" and "lon" columns.
        hexagon_df (gpd.GeoDataFrame): GeoDataFrame with hexagon geometries and associated Hexagon IDs.
    Returns:
        pd.DataFrame: The input DataFrame with an additional "Hexagon_ID" column indicating the Hexagon ID for each ping.
    """

    # Create a column Hexagon_ID with the ID of the hexagon
    df['Hexagon_ID'] = None

    # Iterate through the hotels in the df DataFrame and calculate hotel counts within each hexagon
    for i, ping in df.iterrows():
        if not isinstance(ping['lat'], float):
            # use location of last ping
            found = False
            j=1
            while not found:
                if isinstance(df.loc[i-j,'lat'],float):
                    df.loc[i,'lat']=df.loc[i-j,'lat']
                    df.loc[i,'lng']=df.loc[i-j,'lng']
                elif isinstance(df.loc[i+j,'lat'],float):
                    df.loc[i,'lat']=df.loc[i+j,'lat']
                    df.loc[i,'lng']=df.loc[i+j,'lng']
                else:
                    j+=1
            continue
        resolution=5   
        result = h3.geo_to_h3(ping["lat"], ping["lon"], resolution)
        # print(f'{ping["lat"]},{ping["lon"]}=>{result}')
        if result != 0:
             df.loc[i, 'Hexagon_ID'] = result
    
    return df

def get_adsb_data(data):
    try:
        response = requests.get(data["url"])
        print(f"downloaded {data['url']}")
        json_data = response.json()
        
        df = pd.json_normalize(json_data['aircraft'])

        df = df[df["type"] == 'adsb_icao']
        df['time'] = pd.to_datetime(datetime.fromtimestamp(json_data['now']).strftime("%Y%m%d%H%M%S"))
        df['Hexagon_ID'] = pd.NA
        df['good_bad'] = np.NaN
        df = df[['flight','r','time', 'hex', 'Hexagon_ID', 'alt_baro', 'nic', 'good_bad','lat','lon','type','t','category','version','nac_p','nac_v','track','baro_rate','seen_pos','seen','gs','alt_geom']]	
        df = df[df["type"] == 'adsb_icao']
        df['alt_baro'] = df['alt_baro'].apply(lambda x: 0 if (x == "ground" or x == np.nan ) else float(x))
        df["nic"] = df["nic"].replace(np.NaN,0)

        df = calculate_hexagon_ids(df)

        mapping = {
            "flight": str,
            "r": str,
            "time": "datetime64[ns]",
            "hex": str,
            "Hexagon_ID": str,
            "alt_baro": float,
            "nic": float,
            "good_bad": bool,
            "lat": float,
            "lon": float,
            "type": str,
            "t": str,
            "category": str,
            "version": float,
            "nac_p": float,
            "nac_v": float,
            "track": float,
            "baro_rate": float,
            "seen_pos": float,
            "seen": float,
            "gs": float,
            "alt_geom": float
        }
        df=df.astype(mapping, copy=True)
        df['good_bad'] = df['nic'].apply(lambda x: False if x==np.NaN or x<7 else True)
        df = df[~df["flight"].isnull()]

        #save to pickle
        if not os.path.exists(os.path.dirname(data["csv"])):
            os.makedirs(os.path.dirname(data["csv"]))
        df.to_csv(data["csv"])

    except Exception as e:
        # Handle other exceptions
        print("Get ADSB Data: An error occurred:", e)

def download_data(index, data, length_data_array):

    try:
        # if file exists, do nothing
        if (os.path.isfile(data["csv"])):
            print(f"csv exists {data['csv']}")
            return
        else:
            get_adsb_data(data)
            return

    except Exception as e:
        # Handle other exceptions
        print("Download data: An error occurred:", e)

def join_large_file(folder_path,start,end,file_path):
    if os.path.exists(file_path):
        os.remove(file_path)
    isHeader = True
    with open(file_path, 'w') as outfile:
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            file_datetime = datetime.strptime(file_path, 'csv\\%Y%m%d\\%H%M%SZ.csv')
            print(file_datetime)
            if filename.endswith('.csv') and start <= file_datetime and file_datetime <= end:
                with open(file_path, 'r') as infile:
                    # skip row header
                    if isHeader:
                        outfile.write(infile.read())
                        isHeader = False
                    else:
                        outfile.write(''.join(infile.readlines()[1:]))
                print(f'Joined {file_path}')
        print(f"Saved {file_path}")

def get_flight_data(start_date_time):
    global df1
    global flights_data
    flights = df1["flight"].unique()
    flights = flights[1:]
    display(f"{flights.shape[0]} flights")
    flights_data = pd.DataFrame(flights, columns=['flight']).progress_apply(lambda x: process_flights_data(x['flight']), axis=1)
    display(flights_data.head(10))
    save_df(flights_data, start_date_time, filetype=FileType.FLIGHTS_DATA, parquet=True)
    return flights_data

def process_flights_data(flight):
    global df1
    rows = df1[df1['flight']==flight]
    # print(f"{flight} - {len(rows)} pings")
    bad_count = rows[rows['good_bad'] == False].shape[0]
    total_count = rows.shape[0]
    percentage_bad = bad_count/total_count if total_count!=0 else 0
    result = analyse_data(percentage_bad)
    return pd.Series({'flight': flight, 'bad_count': bad_count, 'total_count': total_count, 'percentage_bad': percentage_bad, 'result': result})

    
def process_data(start_date_time,end_date_time, reload=[False,False,False]):
    delta = dt.timedelta(seconds=5)
    start = start_date_time
    global df1
    global df2
    global flights_data
    file_path = os.path.join('csv', 'joined', f'{start_date_time.strftime("%Y%m%d")}joined.csv')

    if not reload[0]:

        data_array = []
        while start < end_date_time:
            data_array.append({
                "url": f'https://samples.adsbexchange.com/readsb-hist/{start.strftime("%Y/%m/%d")}/{start.strftime("%H%M%S")}Z.json.gz',
                "pickle": os.path.join('csv',start.strftime("%Y%m%d"), f'{start.strftime("%H%M%S")}Z.pkl'),
                "csv": os.path.join('csv',start.strftime("%Y%m%d"), f'{start.strftime("%H%M%S")}Z.csv')
                })
            start += delta

        try:
            with concurrent.futures.ThreadPoolExecutor() as executor:
                l = [len(data_array)] * len(data_array)
                list(executor.map(lambda i: download_data(i, data_array[i], l[i]), range(len(data_array))))
        except Exception as e:
            print("Thread Pool: An error occurred:", e)
            
        join_large_file(os.path.join('csv',start_date_time.strftime("%Y%m%d")),start_date_time,end_date_time,file_path)
        csv_to_parquet(start_date_time, filetype = FileType.JOINED)

    df1 = load_df(start_date_time, filetype=FileType.JOINED, parquet=True)
    
    flights_data = get_flight_data(start_date_time) if not reload[1] else load_df(start_date_time, filetype=FileType.FLIGHTS_DATA, parquet=True)

    df2 = get_hexagon_df(start_date_time) if not reload[2] else load_df(start_date_time, filetype=FileType.HEXBIN, parquet=True)

    return [df1, df2, flights_data]

def get_hexagon_df(start_date_time):
    global df1
    global df2
    df2 = df1.groupby(['Hexagon_ID', 'flight'], as_index=False)[['good_bad','alt_baro','time']].agg(
                            total_count=('good_bad', lambda x: (x == x).sum()),
                            bad_count=('good_bad', lambda x: (x == False).sum()),
                            percentage_bad=('good_bad', lambda x: (x == False).sum()/(x==x).sum()),
                            alt_baro_min=('alt_baro', lambda x: x.min()),
                            alt_baro_max=('alt_baro', lambda x: x.max()),
                            time_min=('time', lambda x: x.min()),
                            time_max=('time', lambda x: x.max())
                            )

    def get_lat(row):
        return h3.h3_to_geo(row['Hexagon_ID'])[0]

    def get_lng(row):
        return h3.h3_to_geo(row['Hexagon_ID'])[1]

    # Apply the function to create new 'lat' and 'lng' columns
    df2['lat'] = df2.apply(get_lat, axis=1)
    df2['lng'] = df2.apply(get_lng, axis=1)

    save_df(df2, start_date_time, filetype=FileType.HEXBIN, parquet=True)
    return df2

class FileType:
    JOINED = "joined"
    SAMPLED = "sampled"
    HEXBIN = "hexbin"
    FLIGHTS_DATA = "flights_data"

def save_df(df, start_date_time, filetype=FileType.JOINED, parquet=False):
    directory = 'joined'
    if filetype == FileType.HEXBIN:
        directory = 'hexbin'
    if parquet:
        file_path = os.path.join('csv', directory , f'{start_date_time.strftime("%Y%m%d")}{filetype}.parquet')
        print(f'Saving parquet {file_path}')
        df.to_parquet(file_path) 
    else:
        file_path = os.path.join('csv', directory , f'{start_date_time.strftime("%Y%m%d")}{filetype}.csv')
        print(f'Saving csv {file_path}')
        if isinstance(df, np.ndarray):
            df = pd.DataFrame(df)
            df.to_csv(file_path) 
        else:
            df.to_csv(file_path) 

def load_df(start_date_time, filetype=FileType.JOINED, parquet=False):
    directory = 'joined'
    if filetype == FileType.HEXBIN:
        directory = 'hexbin'
    if parquet:
        file_path = os.path.join('csv', directory , f'{start_date_time.strftime("%Y%m%d")}{filetype}.parquet')
        print(f'Reading parquet {file_path}')
        return pd.read_parquet(file_path)
    else:
        file_path = os.path.join('csv', directory , f'{start_date_time.strftime("%Y%m%d")}{filetype}.csv')
        return pd.concat([chunk for chunk in tqdm(pd.read_csv(file_path, chunksize=1000000), desc=f'Loading csv {file_path}')])

def csv_to_parquet(start_date_time, filetype=FileType.JOINED):
    chunk_size = 1000000
    parquet_writer = pd.DataFrame()
    directory = 'joined'
    if filetype == FileType.HEXBIN:
        directory = 'hexbin'
    csv_path=os.path.join('csv', directory , f'{start_date_time.strftime("%Y%m%d")}{filetype}.csv')
    parquet_path=os.path.join('csv', directory , f'{start_date_time.strftime("%Y%m%d")}{filetype}.parquet')
    print("Converting csv to parquet")
    for i,chunk in enumerate(pd.read_csv(csv_path, chunksize=chunk_size)):
        sys.stdout.write("Conversion progress: %d   \r" % (i) )
        sys.stdout.flush()
        parquet_writer = pd.concat([parquet_writer,chunk])
    parquet_writer.to_parquet(parquet_path)

def parquet_to_csv(start_date_time, filetype=FileType.JOINED):
    print("Converting parquet to csv")
    df1=load_df(start_date_time, filetype=filetype, parquet=True)
    save_df(df1, start_date_time, filetype=filetype, parquet=False)

class ResultGroup:
    EQUIPMENT_FAILURE = "equipment_failure"
    NORMAL = "normal"
    ANALYSE = "analyse"
    UNKNOWN = "unknown"

def generate_plots(sample_size, group = ResultGroup.ANALYSE):
    global df1
    global flights_data

    if isinstance(df1, type(None)) or isinstance(flights_data, type(None)):
        print("Generate plots: error no data")
        return

    flights = [str(flight) for flight in flights_data[flights_data['result']==group].sample(sample_size)['flight']]
    for flight in flights:
        flight_df = df1[df1['flight']==flight]
        flight_df.set_index("time")

        fig, ax1 = plt.subplots(figsize=(8, 4))
        ax1.plot(flight_df['time'].astype("datetime64[ns]"), flight_df['alt_baro'], 'bx-', label='alt_baro')
        ax1.tick_params('y', colors='b')
        ax1.set_yticks(np.arange(0, 50000, 5000))
        ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%Y %H:%M:%S'))

        ax1.set_xlabel('Date-Time', fontsize=10)
        ax1.set_ylabel('Altitude (ft)', color='b', fontsize=10)

        plt.gcf().autofmt_xdate()

        ax2 = ax1.twinx()
        ax2.plot(flight_df['time'].astype("datetime64[ns]"), flight_df['nic'], 'rx-', label='nic')
        ax2.set_ylabel('NIC', color='g', fontsize=10)
        ax2.tick_params('y', colors='g')
        ax2.set_yticks(np.arange(0, 14, 1))

        plt.title(f'Flight:{flight} Altitude and NIC vs Time  (categorised as {group})', fontsize=10)
        fig.legend(loc="lower right")
        plt.show()
        url = f"https://doc8643.com/aircraft/{flight_df.iloc[0]['t']}"
        print(url)

def analyse_data(percentage_bad):
    upper_limit=0.9
    lower_limit=0.1
    if percentage_bad>upper_limit:
        return "equipment_failure"
    elif percentage_bad<lower_limit:
        return "normal"
    elif percentage_bad>=lower_limit and percentage_bad<=upper_limit:
        return "analyse"
    else: 
        return "unknown"


# hexagon map
def get_hexagon_grid(latitude, longitude, resolution, ring_size):
    """
    Generate a hexagonal grid GeoDataFrame centered around a specified location.
    Parameters:
    - latitude (float): Latitude of the center point.
    - longitude (float): Longitude of the center point.
    - resolution (int): H3 resolution for hexagons.
    - ring_size (int): Number of rings to create around the center hexagon.
    Returns:
    - hexagon_df (geopandas.GeoDataFrame): GeoDataFrame containing hexagons and their geometries.
    """

    global hexagon_df

    # Get the H3 hexagons covering the specified location
    center_h3 = h3.geo_to_h3(latitude, longitude, resolution)
    hexagons = list(h3.k_ring(center_h3, ring_size))  # Convert the set to a list

    # Create a GeoDataFrame with hexagons and their corresponding geometries
    hexagon_geometries = [shapely.geometry.Polygon(h3.h3_to_geo_boundary(hexagon, geo_json=True)) for hexagon in hexagons]
    hexagon_df = gpd.GeoDataFrame({'Hexagon_ID': hexagons, 'geometry': hexagon_geometries})

    
def create_choropleth_map(alpha=0.3, map_style="carto-positron", data="percentage_bad", limits=[0,0.1,0.5,1], lat=home_lat, lon=home_lng):
    """
    Create an interactive choropleth map using Plotly Express.
    Parameters:
    - geojson_df (GeoDataFrame): GeoJSON data containing polygon geometries.
    - data_df (DataFrame): DataFrame containing data to be visualized on the map.
    - alpha (float): Opacity level for the map polygons (0.0 to 1.0).
    - map_style (str): Map style for the Plotly map (e.g., "carto-positron").
    - color_scale (str): Color scale for the choropleth map.
    Returns:
    None
    """
    global df2
    global hexagon_df

    # Hexagon grid around HOME
    get_hexagon_grid(home_lat, home_lng, resolution, ring_size)
    print("hexagons calculated")

    # Merge the GeoJSON data with your DataFrame
    merged_df = hexagon_df.merge(df2, on="Hexagon_ID", how="left")

    # Create a choropleth map using px.choropleth_mapbox
    fig = px.choropleth_mapbox(
        merged_df,
        geojson=merged_df.geometry,
        locations=merged_df.index,  # Use index as locations to avoid duplicate rows
        color=data,
        color_continuous_scale=[[limits[0], f'rgba(0,255,0,{alpha})'],
                                [limits[1], f'rgba(255,255,0,{alpha})'],
                                [limits[2], f'rgba(255,0,0,{alpha})'],
                                [limits[3], f'rgba(255,0,0,{alpha})']],        
        title="GPS Jam Map",
        mapbox_style=map_style,
        center={"lat": home_lat, "lon": home_lng},  # Adjust the center as needed
        zoom=2,
    )

    # Customize the opacity of the hexagons
    fig.update_traces(marker=dict(opacity=alpha))

    # Add hover data for hotel names
    fig.update_traces(customdata=merged_df[["Hexagon_ID","bad_count", "total_count", "percentage_bad", "lat", "lng"]])

    # Define the hover template 
    hover_template = "<b>Hexagon ID:</b> %{customdata[0]}<br><b>Location:</b> %{customdata[4]:.4f},%{customdata[5]:.4f}<br><b>Percentage bad:</b> %{customdata[3]:.3f}<br><b>Total Count:</b> %{customdata[2]}<extra></extra>"
    fig.update_traces(hovertemplate=hover_template)

    # Set margins to 25 on all sides
    fig.update_layout(margin=dict(l=35, r=35, t=45, b=35))
    
    # Adjust the width of the visualization
    fig.update_layout(width=1000) 

    fig.show()


# note that for the free sample, only the first day of each month is available on the adsbexchange
# 4 hours + to download 1 day of files
# 18 mins to reload

# Reload flags are for skipping steps and saving processing time
# reload[0]=True loads data from [date]joined.csv (df1)
# reload[1]=True loads data from [date]flights_data.csv (>45 min) (flights_data)
# reload[2]=True loads data from [date]hexbin.csv (df2)

# [df1, df2, flights_data] = process_data(dt.datetime(2024,5,1,0,0,0),dt.datetime(2024,5,2,0,0,0), reload=[False, False, False])
start_date_time = dt.datetime(2024,6,1,0,0,0)

csv_to_parquet(start_date_time, filetype=FileType.JOINED)


Converting csv to parquet


In [ ]:
# Generate Plots For Sampled Flights, Alt/NIC vs Time
start_date_time = dt.datetime(2024,4,1,0,0,0)
df1 = load_df(start_date_time, filetype=FileType.JOINED, parquet=False)
flights_data = load_df(start_date_time, filetype=FileType.FLIGHTS_DATA, parquet=False)
display(flights_data['result'].value_counts())

# > 0.9: Equipment Failure
# Between 0.1 and 0.9: Analyse
# < 0.1: Normal
# generate_plots(5, group=ResultGroup.NORMAL)
generate_plots(5, group=ResultGroup.ANALYSE)
# generate_plots(5, group=ResultGroup.EQUIPMENT_FAILURE)

In [ ]:
start_date_time = dt.datetime(2024,4,1,0,0,0)
df1 = load_df(start_date_time, filetype=FileType.JOINED, parquet=False)
flights_data = load_df(start_date_time, filetype=FileType.FLIGHTS_DATA, parquet=False)

def generate_correlations(sample_size, group = ResultGroup.ANALYSE):
    global df1
    global flights_data

    if isinstance(df1, type(None)) or isinstance(flights_data, type(None)):
        print("Generate plots: error no data")
        return

    flights = [str(flight) for flight in flights_data[flights_data['result']==group].sample(sample_size)['flight']]
    flights_df=None
    for flight in flights:
        flight_df = df1[df1['flight']==flight]
        flights_df = pd.concat([flights_df,flight_df])    
    
    flights_df.set_index("time")
    print(flights_df.shape[0])
    display(flights_df.head(10))

    corr_matrix = flights_df.corr(method='pearson')
    ax = plt.axes()
    ax.set_title(f'Pearson cofficient for sample size {sample_size}, \ngroup: {group} \nflights: {flights} ')
    sn.heatmap(corr_matrix, annot=True)
    plt.show()

In [ ]:
# Plots Hex Map

start_date_time = dt.datetime(2024,4,1,0,0,0)
df1 = load_df(start_date_time, filetype=FileType.JOINED, parquet=False)
df2 = get_hexagon_df(start_date_time)
display(df2.head(10))

coord = [33.81825, 35.49082]
hexagon_ID = h3.geo_to_h3(coord[0], coord[1], resolution=resolution)
print(f"{coord}->{hexagon_ID}")
create_choropleth_map(data="percentage_bad", limits = [0,0.1,0.5,1], lat=coord[0], lon=coord[1])
# create_choropleth_map(data="bad_count", limits = [0,20,50,100])

flights_in_hex = df1[df1['hexagon_ID']==hexagon_ID]['flight'].unique()

flights_df=None
for flight in flights_in_hex:
        flight_df = df1[df1['flight']==flight and df1['hexagon_ID']==hexagon_ID]
        time_start = flight_df["time"].min()
        time_end = flight_df["time"].max()
        # flight_df2 contains all pings of the flight between the start and end time that the flight is inside the hex. i.e. it includes pings that have null lat/lon values
        flight_df2 = df1[df1['flight']==flight and df1['time'].between(time_start, time_end)]
        flights_df = pd.concat([flights_df,flight_df2])

display(flights_df.head(100))

# given lat/lon -> find hexagon_ID -> filter all flights
# plot flights on hexagon map

# plot Altitude / NIC vs time

# 2024-04-01
# 33.81825, 35.49082
# Label 318/Size 63600
# 30.0842, 31.35159
# Label 281/Size 56200


# 2024-02-01
# 33.81825, 35.49082
# Label 200/Size 40000

# 2024-03-01
# 33.81825, 35.49082
# Label 120/Size 24000



In [ ]:
# DO
# 1. .info() to get data types  or .dtypes  
# 2. .describe() 0> gets mean() count(), 25%, 50%, 75% max min
# 3. .isnull() check for missing values   ->  isnull().sum().sort_values(ascending=False)
# 4. .nunique() -> gets unique values for each field with their counts
# 5. .unique() -> gets a list of unique values in a column
# 6. .value_counts() -> gets unique values for the column with their counts
# 7. for name, group in grouped: -> iterate over groups
# 8. pandas time series functions: Timestamp->DatetimeIndex[] (to_datetime, date_range), Timedelta->TimedeltaIndex[] (to_timedelta, timedelta_range), Period->PeriodIndex[] (Period, period_range), DateOffset (Dateoffset)
# 9. Missing Values: Numeric (pd.nan), String (pd.NA), Time (pd.NaT) -> use .isna(), .notna() to detect missing values

# DON'T DO
# 1. seems like saving as a pickle may corrupt the data, I see a lot of NA, maybe due to the data conversion between types. avoid any data conversion except at the start
# 2. use display() instead of print() 
# 3. .dropna() -> WRONG USE, drops any row or column with a missing value!
# 4. df[df[]] is a filter, not selecting the rows. it is wrong

# flights time Hexagon_ID good_bad lat lon
# problem: some lat lon are NaN because of GPS failure. To assign nearest hexagon_ID based on nearest time with lat lon




In [ ]:
flight = str(flights_data[flights_data['result']=='analyse'].sample(1).iloc[0]['flight'])
print(flight)

flight_df = df1[df1['flight']==flight]
print(flight_df.shape[0])
display(flight_df.head(10))
flight_df.set_index("time")

fig, ax1 = plt.subplots(figsize=(12, 6))
ax1.plot(flight_df['time'], flight_df['alt_baro'], 'b-', label='alt_baro')
ax1.tick_params('y', colors='b')
ax1.set_yticks(np.arange(0, 50000, 5000))

ax1.set_xlabel('Date-Time', fontsize=12)
ax1.set_ylabel('Altitude (ft)', color='b', fontsize=12)

plt.gcf().autofmt_xdate()

ax2 = ax1.twinx()
ax2.plot(flight_df['time'], flight_df['nic'], 'rx', label='nic')
ax2.set_ylabel('NIC', color='g', fontsize=12)
ax2.tick_params('y', colors='g')
ax2.set_yticks(np.arange(0, 14, 1))

plt.title('Altitude and NIC vs Time')
fig.legend(loc="upper right")
plt.show()

In [ ]:
create_choropleth_map(data_df=df2, data="percentage_bad", limits = [0,0.1,0.5,1], )
# create_choropleth_map(data_df=df2, data="bad_count", limits = [0,20,50,100])

In [ ]:

# https://towardsdatascience.com/3-python-packages-for-interactive-data-analysis-3063a201a589
# https://towardsdatascience.com/4-libraries-that-can-perform-eda-in-one-line-of-python-code-b13938a06ae


In [ ]:

# Change in NIC/NAC/SIL indicates an anomaly, which may be due to any reason

# nic: Navigation Integrity Category (2.2.3.2.7.2.6)
# Table 1: NIC value and corresponding size of containment radius
# NIC Containment Radius
# 0 Unknown
# 1 Rc < 37.04km (20nm)
# 2 Rc < 14.816km (8nm)
# 3 Rc < 7.408km (4nm)
# 4 Rc < 3.704km (2nm)
# 5 Rc < 1852m (1nm)
# 6 Rc < 1111.2m (0.6nm)
# Rc < 926m (0.5nm)
# Rc < 555.6m (0.3nm)
# 7 Rc < 370.4m (0.2nm)
# 8 Rc < 185.2m (0.1nm)
# 9 Rc < 75m
# 10 Rc < 25m
# 11 Rc < 7.5m

# Assuming you have already calculated the counts
# counts = df1['nic'].value_counts(dropna=False)
# print(f"List all nic values with counts {counts}")
# # Create bins for the histogram
# bins = np.arange(len(counts) + 1)

# # Plot the histogram with NA on the left
# plt.hist(df1['nic'], bins=bins, align='left',rwidth=0.5)
# plt.yscale('log')
# plt.title('nic')
# plt.show()

print(f"bad {df1[df1['good_bad']=='bad'].shape[0]}")
print(f"total {df1.shape[0]}")
print(f"% bad / total {df1[df1['good_bad']=='bad'].shape[0]/df1.shape[0]*100:.2f}%")
df1[df1['good_bad']=='bad'].head(100)



In [ ]:

# %%
print(f"List all type with counts {df1['type'].value_counts()}")
# List all type with counts type
# adsb_icao         9145 - messages from a Mode S or ADS-B transponder, using a 24-bit ICAO address
# other              725 - IGNORE miscellaneous data received via Basestation / SBS format, quality / source is unknown.
# adsb_icao_nt       524 - IGNORE - messages from an ADS-B equipped “non-transponder” emitter e.g. a ground vehicle, using a 24-bit ICAO address
# mode_s             515 - ModeS data from the planes transponder (no position transmitted)
# adsr_icao          280 - rebroadcast of ADS-B messages originally sent via another data link e.g. UAT, using a 24-bit ICAO address
# tisb_other         256 - traffic information about a non-ADS-B target using a non-ICAO address
# tisb_trackfile     214 - traffic information about a non-ADS-B target using a track/file identifier, typically from primary or Mode A/C radar
# mlat               116 - MLAT, position calculated arrival time differences using multiple receivers, outliers and varying accuracy is expected.
# unknown             49
# tisb_icao           31 - traffic information about a non-ADS-B target identified by a 24-bit ICAO address, e.g. a Mode S target tracked by secondary radar
# adsb_other          17 - messages from an ADS-B transponder using a non-ICAO address, e.g. anonymized address
plt.hist(df1['type'], align='left')
plt.title('type')
plt.xticks(rotation='vertical')
plt.show()


In [ ]:

# %%
# category: emitter category to identify particular aircraft or vehicle classes (values A0 – D7) (2.2.3.2.5.2)
counts = df1['category'].dropna()
# print(f"List all category with counts {counts}")
plt.hist(counts)
plt.title('category')
plt.show()
# A0 : No ADS-B emitter category information. Do not use this emitter category. If no emitter category fits your installation, seek guidance from the FAA as appropriate. A1 : Light (< 15500 lbs) – Any airplane with a maximum takeoff weight less than 15,500 pounds. This includes very light aircraft (light sport aircraft) that do not meet the requirements of 14 CFR § 103.1.
# A2 : Small (15500 to 75000 lbs) – Any airplane with a maximum takeoff weight greater than or equal to15,500 pounds but less than 75,000 pounds.
# A3 : Large (75000 to 300000 lbs) – Any airplane with a maximum takeoff weight greater than or equal to 75,000 pounds but less than 300,000 pounds that does not qualify for the high vortex category.
# A4 :  High vortex large (aircraft such as B-757) – Any airplane with a maximum takeoff weight greater than or equal to 75,000 pounds but less than 300,000 pounds that has been determined to generate a high wake vortex. Currently, the Boeing 757 is the only example.
# A5 : Heavy (> 300000 lbs) – Any airplane with a maximum takeoff weight equal to or above 300,000 pounds.
# A6 : High performance (> 5g acceleration and 400 kts) – Any airplane, regardless of weight, which can maneuver in excess of 5 G’s and maintain true airspeed above 400 knots.
# A7 : Rotorcraft – Any rotorcraft regardless of weight.
# B0 : No ADS-B emitter category information
# B1 : Glider / sailplane – Any glider or sailplane regardless of weight.
# B2 : Lighter-than-air – Any lighter than air (airship or balloon) regardless of weight.
# B3 : Parachutist / skydiver
# B4 : Ultralight / hang-glider / paraglider – A vehicle that meets the requirements of 14 CFR § 103.1. Light sport aircraft should not use the ultralight emitter category unless they meet 14 CFR § 103.1.
# B5 : Reserved
# B6 : Unmanned aerial vehicle – Any unmanned aerial vehicle or unmanned aircraft system regardless of weight.
# B7 : Space / trans-atmospheric vehicle
# C0 : No ADS-B emitter category information
# C1 : Surface vehicle – emergency vehicle
# C2 : Surface vehicle – service vehicle
# C3 : Point obstacle (includes tethered balloons)
# C4 : Cluster obstacle
# C5 : Line obstacle
# C6 : Reserved
# C7 : Reserved
